### Plagarism Check

Blogs are the richest source of information. However, much of the text information is redundant. Tech Mahindra takes up the job of designing a blog retrieval and summarization system where the goal is to retrieve the K least redundant blogs while returning the top-K search results.  

You are a software developer at Tech Mahindra. Your Team is assigned the task of designing the blog search tool. To find the top K results, the team leader decides to use the plagiarism score as a metric for evaluation.  You are being asked to design a prototype for this tool using basic text processing techniques like edit distance and n-gram model considering the processing speed. Write a program for designing the plagiarism check tool. 

Also provide a report describing the design plans, and documentation for your plagiarism check tool. 

In [56]:
import itertools
import re
from collections import Counter
from operator import itemgetter

In [57]:
def tokenizer(words):
    words = re.sub(r'[^\w\s]', '', words)
    words = words.lower()
    tokens = words.split()
    return tokens

In [58]:
#computing edit distance 

def edit_dist(str1, str2):
    m = len(str1)+1
    n = len(str2)+1
    ed = [[0] * (n+1) for _ in range(m+1)]
    for i in range(m):
        for j in range(n):
            if i == 0:
                ed[i][j] = j
            elif j == 0:
                ed[i][j] = i
            elif str1[i-1] == str2[j-1]:
                ed[i][j] = ed[i-1][j-1]
            else:
                ed[i][j] = min(ed[i][j-1], ed[i-1][j], ed[i-1][j-1]) + 1
    return ed[m][n]

In [59]:
#n-grams for input

def n_grams(tokens, n):
    total = []
    for i in range(len(tokens)-n+1):
        ngram = ' '.join(tokens[i:i+n])
        total.append(ngram)
        
    return total

In [60]:
def plagiarism_score(txt1, txt2, n,k):   
    ng1 = n_grams(txt1, n)
    ng2 = n_grams(txt2, n)
   
    ngcount1 = Counter(ng1)
    ngcount2 = Counter(ng2)
    
    intersection = set(ngcount1.keys()) & set(ngcount2.keys())
    score = 0
    
    for ngram in intersection:
        score += min(ngcount1[ngram], ngcount2[ngram])
        
    score /= max(len(ng1), len(ng2))
    edit_distance = edit_dist(txt1, txt2)
    return (score, edit_distance)

In [69]:
if __name__ == '__main__':
    paragraphs = ['The girl with a green umbrella.',
                  'Who are you and how are you doing?',
                  'Here is a sample blog with a little bit of text.',
                  'Here is another sample blog with some more text.',
                  'The girl using a pink umbrella.',
                  'She is walking the dog on the right lane.',
                  'The lady next door is annoying me.',
                  'The girl is walking with a yellow umbrella.',
                  'This is yet another blog for understanding the program.',
                  'This is the final blog of this input.']
    n = 3
    k = 5
    res = []
    for i, (para1, para2) in enumerate(itertools.combinations(paragraphs, 2)):
        score, edit_distance = plagiarism_score(para1, para2, n, k)
        res.append((i+1, score, edit_distance, para1, para2))
    
    res.sort(key=lambda x: x[1])
   
    for i, score, edit_distance, para1, para1 in res:
        print(f'{i}: plagiarism score={score:.5f}, \nedit distance={edit_distance},\ntext1="{para1}", \ntext2="{para2}"\n')

1: plagiarism score=0.00000, 
edit distance=0,
text1="Who are you and how are you doing?", 
text2="This is the final blog of this input."

17: plagiarism score=0.00000, 
edit distance=0,
text1="This is the final blog of this input.", 
text2="This is the final blog of this input."

25: plagiarism score=0.00000, 
edit distance=0,
text1="The girl using a pink umbrella.", 
text2="This is the final blog of this input."

8: plagiarism score=0.01887, 
edit distance=0,
text1="This is yet another blog for understanding the program.", 
text2="This is the final blog of this input."

10: plagiarism score=0.02174, 
edit distance=0,
text1="Here is a sample blog with a little bit of text.", 
text2="This is the final blog of this input."

19: plagiarism score=0.02174, 
edit distance=0,
text1="The girl using a pink umbrella.", 
text2="This is the final blog of this input."

5: plagiarism score=0.02564, 
edit distance=0,
text1="She is walking the dog on the right lane.", 
text2="This is the final blog o

In [70]:
def each_score(p1, p2, ngram=3):
    
    t1 = [p1[i:i+ngram] for i in range(len(p1)-ngram+1)]
    t2 = [p2[i:i+ngram] for i in range(len(p2)-ngram+1)]
    
    intersect = set(t1) & set(t2)
    score = len(intersect) / (len(set(t1)) + len(set(t2)))
    return score

scores = []
for i, p1 in enumerate(paragraphs):
    total_score = 0
    for j, p2 in enumerate(paragraphs):
        if i != j:
            score = each_score(p1, p2)
            total_score += score
    scores.append((p1, total_score))
    
ascending = sorted(scores, key=itemgetter(1))

for paragraph in ascending:
    print(paragraph[0], paragraph[1])

Who are you and how are you doing? 0.2180036567033471
The lady next door is annoying me. 0.5897960243988873
This is the final blog of this input. 0.597604005122748
She is walking the dog on the right lane. 0.6731031337053552
This is yet another blog for understanding the program. 0.6917908643817376
The girl using a pink umbrella. 0.7360369646244921
The girl with a green umbrella. 0.7954583202281356
Here is a sample blog with a little bit of text. 0.818912864415408
Here is another sample blog with some more text. 0.8279816705552
The girl is walking with a yellow umbrella. 1.1255645069810172


In [71]:
for i in range(5):
    paragraph = ascending[i]
    print(paragraph[0])

Who are you and how are you doing?
The lady next door is annoying me.
This is the final blog of this input.
She is walking the dog on the right lane.
This is yet another blog for understanding the program.
